In [1]:
from appgeopy import *
from my_packages import *

In [2]:
from matplotlib.cm import ScalarMappable
from matplotlib.ticker import MaxNLocator
from scipy.interpolate import interpn


# Create a density scatter plot
def density_scatter(x, y, ax=None, sort=True, bins=20, **kwargs):
    if ax is None:
        fig, ax = plt.subplots()

    # 2D histogram to calculate density
    data, x_e, y_e = np.histogram2d(x, y, bins=bins, density=False)

    # Interpolate density values for the scatter points
    z = interpn(
        (0.5 * (x_e[1:] + x_e[:-1]), 0.5 * (y_e[1:] + y_e[:-1])),
        data,
        np.vstack([x, y]).T,
        method="splinef2d",
        bounds_error=False,
    )

    # Handle NaN values (set them to zero)
    z[np.isnan(z)] = 0.0

    # Sort points by density (densest plotted last for visibility)
    if sort:
        idx = z.argsort()
        x, y, z = x[idx], y[idx], z[idx]

    # Create scatter plot, color-coded by density
    scatter = ax.scatter(
        x, y, c=z, cmap="turbo", s=200, edgecolor="none", linewidths=0.75
    )

    # Add color bar
    norm = Normalize(vmin=np.min(z), vmax=np.max(z))
    cbar = ax.get_figure().colorbar(
        ScalarMappable(norm=norm, cmap="turbo"), ax=ax, shrink=0.4, pad=0.01
    )
    cbar.ax.set_ylabel("Points / Unit Area", fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    # Or use MaxNLocator to automatically set tick intervals
    cbar.ax.yaxis.set_major_locator(
        MaxNLocator(integer=True, prune="lower")
    )  # Adjusts tick intervals
    return ax


# ====================================================
def export_scatterplot(
    datatable, folder2save, savename, saveFig=True, showFig=False
):
    # Set the base name for saving figures
    # base = "Leveling_InSAR_scatterplot"
    folder2savefigure = os.path.dirname(filepath)

    cm = 1 / 2.54

    multiplier = 1.5

    fig = plt.figure(figsize=(10 * multiplier, 10 * multiplier))
    ax = fig.add_subplot(1, 1, 1)

    ax.grid(axis="both", which="major", color="lightgrey", alpha=0.5)
    top_thres = 2
    bot_thres = -8

    x1, y1 = [top_thres, bot_thres], [top_thres, bot_thres]
    ax.plot(x1, y1, linestyle="--", color="gray", linewidth=4, zorder=1)

    # ax.set_xlim(bot_thres, top_thres)
    # ax.set_ylim(bot_thres, top_thres)

    ax.set_aspect(aspect="equal", adjustable="box")

    x_axis_object = "InSAR_cm_yr"
    y_axis_object = "Leveling_cm_yr"
    x_axis_label = r"${\nu}_{InSAR}$ (cm/year)"
    y_axis_label = r"${\nu}_{leveling}$ (cm/year)"

    scatter_points = [
        (psc, lev) for psc, lev in zip(df[x_axis_object], df[y_axis_object])
    ]

    # scatter_plot = ax.scatter(*zip(*scatter_points), s=200, facecolors="none", edgecolors="None", marker="o")

    x = df[x_axis_object]
    y = df[y_axis_object]

    # Call the density scatter plot function
    density_scatter(x, y, ax=ax, bins=50)

    ax.set_xlabel(x_axis_label, fontsize=45, labelpad=15)
    ax.set_ylabel(y_axis_label, fontsize=45, labelpad=15)

    mean_abs_err = np.mean(np.abs(x - y))
    rmse = np.sqrt(np.mean((x - y) ** 2))
    m, c, r, p, se1 = stats.linregress(x, y)

    major_base = 1
    minor_base = 0.5

    x_major_loc = plticker.MultipleLocator(base=major_base)
    x_minor_loc = plticker.MultipleLocator(base=minor_base)
    ax.xaxis.set_major_locator(x_major_loc)
    ax.xaxis.set_minor_locator(x_minor_loc)

    y_major_loc = plticker.MultipleLocator(base=major_base)
    y_minor_loc = plticker.MultipleLocator(base=minor_base)
    ax.yaxis.set_major_locator(y_major_loc)
    ax.yaxis.set_minor_locator(y_minor_loc)

    ax.tick_params(
        axis="y",
        which="major",
        labelsize=45,
        direction="out",
        length=16,
        width=2,
    )
    ax.tick_params(
        axis="y",
        which="minor",
        labelsize=45,
        direction="out",
        length=12,
        width=2,
    )

    ax.tick_params(
        axis="x",
        which="major",
        labelsize=45,
        direction="out",
        length=16,
        width=2,
    )
    ax.tick_params(
        axis="x",
        which="minor",
        labelsize=45,
        direction="out",
        length=12,
        width=2,
    )

    ax.text(
        0.075,
        0.95,
        f"MAE = {mean_abs_err:.2f} cm/year",
        transform=plt.gca().transAxes,
        fontweight="bold",
        fontsize=40,
    )
    ax.text(
        0.075,
        0.91,
        f"RMSE = {rmse:.2f} cm/year",
        transform=plt.gca().transAxes,
        fontweight="bold",
        fontsize=40,
    )

    ax.text(
        0.075,
        0.87,
        f"r = {r:.2f}",
        transform=plt.gca().transAxes,
        fontweight="bold",
        fontsize=40,
    )

    ax.spines[["right", "top"]].set_visible(False)

    ax.axhline(0, color="lightgrey")
    ax.axvline(0, color="lightgrey")

    fig.tight_layout()

    # for i in range(1, 1001):
    #     savename = os.path.join(folder2savefigure, base + "_{}.png".format(str(i).zfill(3)))
    #     if os.path.exists(savename):
    #         continue
    #     else:
    #         break

    if saveFig:
        plt.savefig(
            os.path.join(folder2save, savename),
            dpi=600,
            transparent=False,
            facecolor="w",
            edgecolor="w",
            bbox_inches="tight",
        )
    plt.close()

In [3]:
buffer_radius = 100
folder2savefig = (
    r"E:\001_InSAR_Project\1100_CHOUSHUI_2025\CRFP_full_to2025\figs"
)

# for file_folder in tqdm(
#     glob(
#         os.path.join(
#             r"E:\001_InSAR_Project\1100_CHOUSHUI_2025\CRFP_full_2016_2024",
#             "asc_desc_run*",
#         )
#     )
# ):
file_folder = r"E:\001_InSAR_Project\1100_CHOUSHUI_2025\CRFP_full_to2025\asc_desc_run004BEB_monthly_fitted"
try:
    # Load data from Excel
    file_name = f"Leveling_InSAR_inBuffer_{buffer_radius}m.xlsx"
    filepath = os.path.join(file_folder, file_name)
    fig_savename = os.path.basename(file_folder) + ".png"

    df = pd.read_excel(filepath)
    df["SqrDiff"] = (df["Leveling_cm_yr"] - df["InSAR_cm_yr"]) ** 2

    lower_bound = np.mean(df["SqrDiff"]) - 3 * np.std(df["SqrDiff"])
    upper_bound = np.mean(df["SqrDiff"]) + 3 * np.std(df["SqrDiff"])

    df = df.query("SqrDiff>=@lower_bound & SqrDiff<=@upper_bound").reset_index(
        drop=True
    )

    export_scatterplot(
        datatable=df, folder2save=folder2savefig, savename=fig_savename
    )
except Exception as e:
    print(file_folder, e)
    pass